The function does predictions on all images in a specified directory (sdir) and prints out the results.  
It has two modes of operation. In the non_averaging mode a seperate prediction is made for each image   
and the predicted class and probability is printed out for each image.  
In the averaging mode a prediction is made on each image and the probabilities are stored.  
When the predictions for all images are completed the probabilites are summed for each class.    
The class having the highest probability sum is then select as the predicted class for all images.  
The predicted class and its averaged probability is printed out  
Lets say you have 10 images of the face of the same child and you want to get a prediction if the child is autistic.    
You can get 10 individual prediction and look at each prediction. However you will get a much higher confidence    
prediction using the predictor in the averaging mode for the 10 images. Of course you can just predict on one image of a childs face  
but you get a much higher confidence if you use multiple facial images of the child and use the predictor in the averaging mode.
  
 The parameters in the function are:   
  * model_path is the  full path to the trained model 
  * txt_path is the full path to the classes.txt file generated by the autism.ipynb notebook when it is run to produce the trained model.    
      The classes.txt file is of the form class0, class1, ...classN. For the case of the autistic dataset the file appears as  
      autistic,non_autistic.
  * img_size is a tupple (height, width) representing the image shape used to train the model   
  * mode is a string. If mode=='ave' the predictor provides a single prediction of the class and class probability. If mode is any    
      other string value the predictor provides an individual prediction for each image and returns a dataframe with columns  
      filename, class, probability. Each row of the dataframe contain the image file name, the class predicted and the probability    
      of the prediction.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import numpy as np

In [5]:
def predictor(sdir, model_path, txt_path, img_size,  mode):
    with open(txt_path, 'r') as f:
        content=f.read() 
    split=content.split(',')
    classes=[split[i] for i in range(len(split))]
    num_classes=len(classes) 
    print("Loading the model. This may take several seconds")
    model = tf.keras.models.load_model(model_path)
    print ('model is loaded')
    flist=sorted(os.listdir(sdir))
    print ('predicting the class of the ', len(flist), ' files in ', sdir)   
    filepaths=[]       
    sum_list=[0 for i in range( num_classes )]    
    for i, f in enumerate(flist):
        fpath=os.path.join(sdir,f)
        filepaths.append(fpath)
    Fseries=pd.Series(filepaths, name ='filepaths')
    df=pd.concat([Fseries], axis=1)
    gen=tf.keras.preprocessing.image.ImageDataGenerator().flow_from_dataframe(df, x_col='filepaths', y_col=None, target_size=img_size, class_mode=None, shuffle=False)
    preds=model.predict(gen)
    cclass=[] # list of predicted classes
    pclass=[] # list of class probabilities   
    for i, p in enumerate(preds):   #iterate through the predictions     
        for  j, cp in enumerate(p): # update the sum_list by adding in the probabilities of each class
            sum_list[j] +=cp        
        index=np.argmax(p) # find the index of the value in p that has the highest probability
        klass=classes[index] # get the class with the highest probability
        cclass.append(klass)
        pclass.append(p[index]* 100)
    Pseries=pd.Series(pclass, name='% Probability')
    Cseries=pd.Series(cclass, name='Predicted Class')
    Fseries=pd.Series(flist, name='Filename')                                                                          
    pdf=pd.concat([Fseries, Cseries, Pseries], axis=1)  # dataframe of the form Filename, Predicted Class, % Probability      
    ave_index=np.argmax(sum_list)
    ave_class=classes[ave_index]    
    ave_p=sum_list[ave_index]* 100/len(flist)    
    if mode == 'ave':
        print (f'The class prediction for all images is {ave_class} with a probability of {ave_p:6.2f} %')
        return ave_class, ave_p                                             
    else:
        print (pdf)
        return pdf                                                                   
   

In [10]:
# example in non averaging mode-individual predicts ae made and a dataframe is returned
txt_path=r"C:\Users\TI\Documents\fyp\DataSets\image dataset of autism\classes.txt"
sdir=r"C:\Users\TI\Documents\fyp\DataSets\image dataset of autism\tests_non_autistic"
img_size=(200,200)
mode=''
#model_path=r"C:\Temp\Autism\autism-vgg19_77.0.h5"
model_path=r"C:\Temp\Autism\autism-B3_84.5.h5"
df= predictor(sdir, model_path, txt_path, img_size, mode)

Loading the model. This may take several seconds
model is loaded
predicting the class of the  5  files in  C:\Users\TI\Documents\fyp\DataSets\image dataset of autism\tests_non_autistic
Found 5 validated image filenames.
1/1 [==============================] - 1s 1s/step
  Filename Predicted Class  % Probability
0  006.jpg    non_autistic      63.518983
1  007.jpg    non_autistic      99.965513
2  008.jpg    non_autistic      72.807181
3  009.jpg        autistic      99.854940
4  010.jpg    non_autistic      99.998379


In [11]:
# example on same data of averaging mode - 1 prediction is made for all images. returns the predicted class and its probability
mode='ave'
ave_class, ave_p  =predictor(sdir, model_path, txt_path, img_size,  mode)

Loading the model. This may take several seconds
model is loaded
predicting the class of the  5  files in  C:\Users\TI\Documents\fyp\DataSets\image dataset of autism\tests_non_autistic
Found 5 validated image filenames.
1/1 [==============================] - 1s 1s/step
The class prediction for all images is non_autistic with a probability of  67.29 %
